In [26]:
# 필수 라이브러리 임포트
from typing import Tuple
import matplotlib
import meep as mp
import nlopt
import numpy as np
import os

# matplotlib의 백엔드를 "agg"로 설정하여 이미지를 파일로 저장 가능하게 합니다.
matplotlib.use("agg")
import matplotlib.pyplot as plt
 
# Lorentzian 함수 정의: Lorentzian 파라미터를 사용하여 복소 유전율 프로파일을 계산
def lorentzfunc(p: np.ndarray, x: np.ndarray) -> np.ndarray:
    N = len(p) // 3  # 파라미터가 3개씩 묶인 Lorentzian 항의 개수
    y = np.zeros(len(x))  # 결과값을 저장할 배열
    for n in range(N):
        A_n = p[3 * n + 0]  # 첫 번째 파라미터 (진폭)
        x_n = p[3 * n + 1]  # 두 번째 파라미터 (중심 주파수)
        g_n = p[3 * n + 2]  # 세 번째 파라미터 (감쇠율)
        y = y + A_n / (np.square(x_n) - np.square(x) - 1j * x * g_n)  # Lorentzian 함수 계산
    return y

# 잔차 함수 정의: 실제 값과 Lorentzian 모델 간의 차이를 계산하고 그라디언트도 반환
def lorentzerr(p: np.ndarray, x: np.ndarray, y: np.ndarray, grad: np.ndarray) -> float:
    N = len(p) // 3  # Lorentzian 항의 개수
    yp = lorentzfunc(p, x)  # 예측된 유전율 프로파일 계산
    val = np.sum(np.square(abs(y - yp)))  # 실제 값과 예측 값의 차이(L2 norm)

    # 그라디언트 계산
    for n in range(N):
        A_n = p[3 * n + 0]
        x_n = p[3 * n + 1]
        g_n = p[3 * n + 2]
        d = 1 / (np.square(x_n) - np.square(x) - 1j * x * g_n)  # Lorentzian 함수의 도함수
        if grad.size > 0:
            grad[3 * n + 0] = 2 * np.real(np.dot(np.conj(yp - y), d))  # 진폭에 대한 그라디언트
            grad[3 * n + 1] = -4 * x_n * A_n * np.real(np.dot(np.conj(yp - y), np.square(d)))  # 중심 주파수에 대한 그라디언트
            grad[3 * n + 2] = -2 * A_n * np.imag(np.dot(np.conj(yp - y), x * np.square(d)))  # 감쇠율에 대한 그라디언트
    return val  # 최종 오차 반환

# Lorentzian 파라미터 최적화 함수 정의
def lorentzfit(
    p0: np.ndarray,
    x: np.ndarray,
    y: np.ndarray,
    alg=nlopt.LD_LBFGS,
    tol: float = 1e-25,
    maxeval: float = 10000,
) -> Tuple[np.ndarray, float]:
    # NLopt 최적화 설정
    opt = nlopt.opt(alg, len(p0))  # 최적화 알고리즘 설정 (LD_LBFGS는 LBFGS 알고리즘)
    opt.set_ftol_rel(tol)  # 상대 오차 허용 범위 설정
    opt.set_maxeval(maxeval)  # 최대 반복 횟수 설정
    opt.set_lower_bounds(np.zeros(len(p0)))  # 하한값 설정
    opt.set_upper_bounds(float("inf") * np.ones(len(p0)))  # 상한값 설정
    opt.set_min_objective(lambda p, grad: lorentzerr(p, x, y, grad))  # 목표 함수 설정
    local_opt = nlopt.opt(nlopt.LD_LBFGS, len(p0))  # 로컬 최적화 설정
    local_opt.set_ftol_rel(1e-10)  # 로컬 최적화 상대 오차 설정
    local_opt.set_xtol_rel(1e-8)  # 로컬 최적화 X 변화 상대 오차 설정
    opt.set_local_optimizer(local_opt)  # 로컬 최적화 설정 추가
    popt = opt.optimize(p0)  # 최적화 실행
    minf = opt.last_optimum_value()  # 마지막 최적화 값 반환
    return popt, minf  # 최적화된 파라미터와 최소 오차 반환

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import meep as mp
import nlopt
import os

# --- 1) 원본 permittivity 데이터 로드 (μm 단위) ---
csv_path      = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/SiO2/Material data/sio2_material_data.csv"
data          = np.genfromtxt(csv_path, delimiter=",")
wl            = data[:, 0]                    # 파장 (μm)
eps_total     = data[:, 1] + 1j*data[:, 2]     # 이미 permittivity

# --- 2) 비교용 CSV 파일 로드 (same 형식: wl, Re(eps), Im(eps)) ---
compare_path  = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/SiO2/lumerical_sio2_eps.csv"
cmp_data      = np.genfromtxt(compare_path, delimiter=",")
wl_cmp        = cmp_data[:, 0]
eps_cmp       = cmp_data[:, 1] + 1j*cmp_data[:, 2]

# --- 3) 피팅 범위 설정 ---
# --- 공통 설정 (위에서 로드한 wl_red, freqs_red, eps_fit_target_raw 등 사용) ---
wl_min, wl_max = 0.39, 0.73
mask = (wl >= wl_min) & (wl <= wl_max)
wl_red    = wl[mask]
freqs_red = 1.0 / wl_red
eps_total_red = eps_total[mask]
eps_fit_target_raw = eps_total_red   # eps_inf 빼기 전 원본

# 피팅 함수 (eps_inf 고정)
def fit_for_eps_inf(eps_inf):
    eps_fit_target = eps_fit_target_raw - eps_inf
    num_L, num_rep = 2, 100
    best_err = np.inf
    best_p  = None
    for _ in range(num_rep):
        # 초기 파라미터 (σ, ω, γ)
        p0 = []
        for _ in range(num_L):
            sigma0 = 10**np.random.uniform(0,2)  # [1,100]
            omega0 = np.random.uniform(1.37,2.63)
            gamma0 = 10**np.random.uniform(-2,0) # [1e-2,1]
            p0.extend([sigma0, omega0, gamma0])
        p_opt, err = lorentzfit(p0, freqs_red, eps_fit_target,
                                nlopt.LD_MMA, 1e-25, 200000)
        if err < best_err:
            best_err = err
            best_p   = p_opt
    return best_err, best_p

# ε∞ 스윕
eps_inf_vals = np.linspace(1, 1.1, 1)
results = []
for e_inf in eps_inf_vals:
    err, p_opt = fit_for_eps_inf(e_inf)
    results.append((e_inf, err, p_opt))

# 최적 ε∞ 선택
eps_inf_best, err_best, p_opt_best = min(results, key=lambda x: x[1])
print("Best eps_inf:", eps_inf_best, "with error:", err_best)
num_L = 2
# 최적 파라미터로 다시 medium 생성
suscepts = []
for j in range(num_L):
    ω = p_opt[3*j + 1]
    γ = p_opt[3*j + 2]
    if ω == 0:
        σ = p_opt[3*j + 0]
        suscepts.append(
            mp.DrudeSusceptibility(
                frequency=1.0,
                gamma=γ,
                sigma=σ
            )
        )
    else:
        σ = p_opt[3*j + 0] / ω**2
        suscepts.append(
            mp.LorentzianSusceptibility(
                frequency=ω,
                gamma=γ,
                sigma=σ
            )
        )
material = mp.Medium(epsilon=eps_inf_best, E_susceptibilities=suscepts)

# 새 모델 커브 계산
eps_model_new = np.array([material.epsilon(f)[0][0] for f in freqs_red])

# --- 7) 플롯 그리기 ---
fig, axes = plt.subplots(ncols=2, figsize=(10,4))

# 레이블 폰트 크기
label_fs = 20  
# 범례 폰트 크기
legend_fs = 14

# Re(ε) 플롯: 원본, Meep fit, 비교 CSV
axes[0].plot(wl_red, np.real(eps_total[mask]), 'gs', label="Material data")
axes[0].plot(wl_red, np.real(eps_model_new),    'b-',  label="Meep fit")
axes[0].plot(wl_cmp, np.real(eps_cmp),      'r-',  label="Lumerical fit")
axes[0].set_xlabel("Wavelength (μm)", fontsize=label_fs)
axes[0].set_ylabel("Re(ε)",             fontsize=label_fs)
axes[0].grid(True)
axes[0].legend(fontsize=legend_fs)

# Im(ε) 플롯: 원본, Meep fit, 비교 CSV
axes[1].plot(wl_red, np.imag(eps_total[mask]), 'gs', label="Material data")
axes[1].plot(wl_red, np.imag(eps_model_new),       'b-',  label="Meep fit")
axes[1].plot(wl_cmp, np.imag(eps_cmp),         'r-',  label="Lumerical fit")
axes[1].set_xlabel("Wavelength (μm)", fontsize=label_fs)
axes[1].set_ylabel("Im(ε)",             fontsize=label_fs)
axes[1].grid(True)
axes[1].legend(fontsize=legend_fs)
fig.suptitle("Drude–Lorentzian Fit(SiO2) - Meep vs Lumerical", fontsize=label_fs)

# --- 8) 플롯 저장 ---
out_fname = "eps_fit_with_compare.png"
fig.savefig(out_fname, dpi=150, bbox_inches="tight")
print("Saved:", os.path.abspath(out_fname))


FileNotFoundError: /home/m2ni/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/SiO2/Material data/sio2_material_data.csv not found.

In [34]:
def fit_and_save_eps_fit(material_csv, compare_csv, output_filename):
    # 1) 원본 permittivity 데이터 로드 (μm 단위)
    data = np.genfromtxt(material_csv, delimiter=",")
    wl = data[:, 0]
    eps_total = data[:, 1] + 1j * data[:, 2]

    # 2) 비교용 CSV 파일 로드
    cmp_data = np.genfromtxt(compare_csv, delimiter=",")
    wl_cmp = cmp_data[:, 0]
    eps_cmp = cmp_data[:, 1] + 1j * cmp_data[:, 2]

    # 3) 피팅 범위 설정
    eps_inf = 1
    wl_min, wl_max = 0.37, 0.71
    mask = (wl >= wl_min) & (wl <= wl_max)
    wl_red = wl[mask]
    freqs_red = 1.0 / wl_red
    eps_fit_target = eps_total[mask] - eps_inf

    # 4) Lorentz-Drude 피팅
    num_L, num_rep = 3, 30
    ps = np.zeros((num_rep, 3 * num_L))
    errs = np.zeros(num_rep)
    for i in range(num_rep):
        p0 = [10**np.random.uniform(2, 5) for _ in range(3 * num_L)]
        ps[i], errs[i] = lorentzfit(p0, freqs_red, eps_fit_target,
                                    nlopt.LD_MMA, 1e-25, 50000)
    best = np.argmin(errs)
    p_opt = ps[best]

    # 5) Meep Medium 생성
    suscepts = []
    for j in range(num_L):
        ω = p_opt[3*j + 1]
        γ = p_opt[3*j + 2]
        if ω == 0:
            σ = p_opt[3*j + 0]
            suscepts.append(mp.DrudeSusceptibility(frequency=1.0, gamma=γ, sigma=σ))
        else:
            σ = p_opt[3*j + 0] / ω**2
            suscepts.append(mp.LorentzianSusceptibility(frequency=ω, gamma=γ, sigma=σ))
    material = mp.Medium(epsilon=eps_inf, E_susceptibilities=suscepts)

    # 6) 모델 ε 계산
    eps_model = np.array([material.epsilon(f)[0][0] for f in freqs_red])

    # 7) 플롯 그리기
    fig, axes = plt.subplots(ncols=2, figsize=(10, 4))
    label_fs = 20
    legend_fs = 14

    # Re(ε)
    axes[0].plot(wl_red, np.real(eps_total[mask]), 'gs', label="Material data")
    axes[0].plot(wl_red, np.real(eps_model), 'b-', label="Meep fit")
    axes[0].plot(wl_cmp, np.real(eps_cmp), 'r-', label="Lumerical fit")
    axes[0].set_xlabel("Wavelength (μm)", fontsize=label_fs)
    axes[0].set_ylabel("Re(ε)", fontsize=label_fs)
    axes[0].grid(True)
    axes[0].legend(fontsize=legend_fs)

    # Im(ε)
    axes[1].plot(wl_red, np.imag(eps_total[mask]), 'gs', label="Material data")
    axes[1].plot(wl_red, np.imag(eps_model), 'b-', label="Meep fit")
    axes[1].plot(wl_cmp, np.imag(eps_cmp), 'r-', label="Lumerical fit")
    axes[1].set_xlabel("Wavelength (μm)", fontsize=label_fs)
    axes[1].set_ylabel("Im(ε)", fontsize=label_fs)
    axes[1].grid(True)
    axes[1].legend(fontsize=legend_fs)

    fig.suptitle("Drude–Lorentzian Fit - Meep vs Lumerical", fontsize=label_fs)

    # 8) 플롯 저장
    fig.savefig(output_filename, dpi=150, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved: {os.path.abspath(output_filename)}")

In [42]:
# al_path = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/Al/Material data/al_material_data.csv"
# andp_path = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/aNDP/lumerical_aNDP_eps.csv"
sio2_path = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/SiO2/lumerical_sio2_eps.csv"

# al_fdtd_path = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/Al/merged_al_eps.csv"
# andp_fdtd_path = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/aNDP/lumerical_aNDP_eps.csv"
sio2_fdtd_path = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/SiO2/lumerical_sio2_eps.csv"

In [43]:
# fit_and_save_eps_fit(al_path, al_fdtd_path, "eps_fit_al_compare.png")
# fit_and_save_eps_fit(andp_path, andp_fdtd_path, "eps_fit_andp_compare.png")
fit_and_save_eps_fit(sio2_path, sio2_fdtd_path, "eps_fit_sio2_compare.png")

Saved: /home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/eps_fit_sio2_compare.png


In [101]:
def Material_fit(Material_data_csv=".", 
                   eps_inf_min=1.1, eps_inf_max=1.2, eps_inf_linespace = 1,
                   wl_min=0.39, wl_max=0.73,
                   num_L=2, num_rep = 100):    
    Material_data_csv      = "/home/m2ni/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/SiO2/Material data/sio2_material_data.csv"
    data          = np.genfromtxt(Material_data_csv, delimiter=",")
    wl            = data[:, 0]                    # 파장 (μm)
    eps_total     = data[:, 1] + 1j*data[:, 2]     # 이미 permittivity

    # --- 3) 피팅 범위 설정 ---
    # --- 공통 설정 (위에서 로드한 wl_red, freqs_red, eps_fit_target_raw 등 사용) ---
    wl_min, wl_max = 0.39, 0.73
    mask = (wl >= wl_min) & (wl <= wl_max)
    wl_red    = wl[mask]
    freqs_red = 1.0 / wl_red
    eps_total_red = eps_total[mask]
    eps_fit_target_raw = eps_total_red   # eps_inf 빼기 전 원본

    # 피팅 함수 (eps_inf 고정)
    def fit_for_eps_inf(eps_inf):
        eps_fit_target = eps_fit_target_raw - eps_inf
        num_L, num_rep = 2, 100
        best_err = np.inf
        best_p  = None
        for _ in range(num_rep):
            # 초기 파라미터 (σ, ω, γ)
            p0 = []
            for _ in range(num_L):
                sigma0 = 10**np.random.uniform(0,2)  # [1,100]
                omega0 = np.random.uniform(1.37,2.63)
                gamma0 = 10**np.random.uniform(-2,0) # [1e-2,1]
                p0.extend([sigma0, omega0, gamma0])
            p_opt, err = lorentzfit(p0, freqs_red, eps_fit_target,
                                    nlopt.LD_MMA, 1e-25, 200000)
            if err < best_err:
                best_err = err
                best_p   = p_opt
        return best_err, best_p

    # ε∞ 스윕
    eps_inf_vals = np.linspace(eps_inf_min, eps_inf_max, eps_inf_linespace)
    results = []
    for e_inf in eps_inf_vals:
        err, p_opt = fit_for_eps_inf(e_inf)
        results.append((e_inf, err, p_opt))

    # 최적 ε∞ 선택
    eps_inf_best, err_best, p_opt_best = min(results, key=lambda x: x[1])
    print("Best eps_inf:", eps_inf_best, "with error:", err_best)
    num_L = 2
    # 최적 파라미터로 다시 medium 생성
    suscepts = []
    for j in range(num_L):
        ω = p_opt[3*j + 1]
        γ = p_opt[3*j + 2]
        if ω == 0:
            σ = p_opt[3*j + 0]
            suscepts.append(
                mp.DrudeSusceptibility(
                    frequency=1.0,
                    gamma=γ,
                    sigma=σ
                )
            )
        else:
            σ = p_opt[3*j + 0] / ω**2
            suscepts.append(
                mp.LorentzianSusceptibility(
                    frequency=ω,
                    gamma=γ,
                    sigma=σ
                )
            )
    return eps_inf_best, suscepts

아래는 지우기 전 fitting 코드

In [ ]:
def Material_fit(Material_data_csv=".", 
                   eps_inf=1.1,
                   fit_wl_min=0.4, fit_wl_max=0.7,
                   num_lorentzians=2, iteration=50):    
        data          = np.genfromtxt(Material_data_csv, delimiter=",")
        wl            = data[:, 0]                    # 파장 (μm)
        eps_total     = data[:, 1] + 1j*data[:, 2]     # 이미 permittivity

        # --- 3) 피팅 범위 설정 ---
        eps_inf   = 1.1
        wl_min, wl_max = 0.4, 0.7
        mask      = (wl >= wl_min) & (wl <= wl_max)
        wl_red    = wl[mask]
        freqs_red = 1.0 / wl_red
        eps_fit_target = eps_total[mask] - eps_inf

        # --- 4) Lorentz-Drude 피팅 (생략: 기존 lorentzfit 사용) ---
        num_L, num_rep = num_lorentzians, iteration

        ps   = np.zeros((num_rep, 3*num_L))
        errs = np.zeros(num_rep)
        for i in range(num_rep):
            p0 = [10**np.random.uniform(2,5) for _ in range(3*num_L)]
            ps[i], errs[i] = lorentzfit(p0, freqs_red, eps_fit_target,
                                        nlopt.LD_MMA, 1e-25, 50000)
            print(f"Iteration {i:3d}, error: {errs[i]:.6f}")                                   
        best = np.argmin(errs)
        p_opt = ps[best]
        print(f"Optimal error: {errs[best]:.6f}")

        # --- 5) Meep Medium 생성 (수정된 부분) ---
        suscept = []
        for j in range(num_L):
            ω = p_opt[3*j + 1]
            γ = p_opt[3*j + 2]
            if ω == 0:
                σ = p_opt[3*j + 0]
                suscept.append(
                    mp.DrudeSusceptibility(
                        frequency=1.0,
                        gamma=γ,
                        sigma=σ
                    )
                )
            else:
                σ = p_opt[3*j + 0] / ω**2
                suscept.append(
                    mp.LorentzianSusceptibility(
                        frequency=ω,
                        gamma=γ,
                        sigma=σ
                    )
                )
    
        return eps_inf, suscept
    
sio2_data_path = "/home/m2ni/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/SiO2/merged_sio2_eps.csv"
al_data_path   = "/home/m2ni/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/Al/merged_al_eps.csv"
andp_data_path = "/home/m2ni/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/aNDP/merged_aNDP_eps.csv"

eps_inf_sio2, suscept_sio2, sio2_popt = Material_fit(fit_wl_min=0.35, fit_wl_max=0.7, Material_data_csv=sio2_data_path)
eps_inf_al, suscept_al, al_popt = Material_fit(Material_data_csv=al_data_path)
eps_inf_andp, suscept_andp, andp_popt = Material_fit(Material_data_csv=andp_data_path)